##**Import Libraries**

In [1]:
# Reloads the autoreload extension to automatically reload modules before code execution
%reload_ext autoreload

# Sets autoreload to the most aggressive mode (reload all modules, both from import statements and interactive use)
%autoreload 2

# Ensures that Matplotlib visualizations are displayed directly in the notebook cells
%matplotlib inline

In [2]:
# Installing the the specified Python packages
! pip install -q transformers[sentencepiece] fastai ohmeow-blurr nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
! pip install -q onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.5 MB/s eta 0:00:00


In [4]:
! pip install onnx onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.1 MB/s eta 0:00:00


In [5]:
# Importing necessary libraries and modules

import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [6]:
from tqdm.notebook import tqdm
import numpy as np

In [7]:
# Mounting Google Drive and specifying the mount point as '/content/drive'

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/Datasets/NLP_Project

/content/drive/MyDrive/Datasets/NLP_Project


##**Data**

In [9]:
# Reading a CSV file named "movie_data.csv" into a Pandas DataFrame
df = pd.read_csv("movie_data.csv")

# Displaying the first few rows of the DataFrame using the head() method
df.head()

,Title,Description,Genres
0,Rebel Moon: Part One - A Child of Fire,"When a peaceful settlement on the edge of a distant moon finds itself threatened by the armies of a tyrannical ruling force, a mysterious stranger living among its villagers becomes their best hope for survival.","Action, Adventure, Drama"
1,Saltburn,"A student at Oxford University finds himself drawn into the world of a charming and aristocratic classmate, who invites him to his eccentric family's sprawling estate for a summer never to be forgotten.","Comedy, Drama, Thriller"
2,Leave the World Behind,"A family's getaway to a luxurious rental home takes an ominous turn when a cyberattack knocks out their devices, and two strangers appear at their door.","Drama, Mystery, Thriller"
3,Wonka,"With dreams of opening a shop in a city renowned for its chocolate, a young and poor Willy Wonka discovers that the industry is run by a cartel of greedy chocolatiers.","Adventure, Comedy, Family"
4,Aquaman and the Lost Kingdom,"Black Manta seeks revenge on Aquaman for his father's death. Wielding the Black Trident's power, he becomes a formidable foe. To defend Atlantis, Aquaman forges an alliance with his imprisoned brother. They must protect the kingdom.","Action, Adventure, Fantasy"


In [10]:
# Drop rows with missing values and reset the index
df = df.dropna().reset_index(drop=True)

# Drop duplicate rows from the DataFrame
df = df.drop_duplicates()

# Display the shape of the DataFrame (number of rows and columns)
print(df.shape)

(12874, 3)


In [11]:
# Extract 'Genres' column values as a list
genres_list = df['Genres'].tolist()

# Clean and preprocess genre names
cleaned_genres_list = []
for genres in genres_list:
    # Split genres and remove extra spaces and dashes
    genre_list = [genre.strip().replace('-', '') for genre in genres.split(',')]

    # Replace 'Sci' or 'Fi' with 'Sci-fi' in the genre list
    genre_list = ['Sci-fi' if genre.lower() == 'sci' or genre.lower() == 'fi' else genre for genre in genre_list]

    # Join the cleaned genre names back into a comma-separated string
    cleaned_genres_list.append(', '.join(genre_list))

# Update the 'Genres' column in the DataFrame with cleaned genre names
df['genres'] = cleaned_genres_list

In [12]:
# Drop the 'Genres' column from the DataFrame
df.drop(columns=['Genres'], inplace=True)

**Data Check**

In [13]:
# Count the occurrences of each genre in the cleaned genre list
genre_count = {}
for genres in cleaned_genres_list:
    genre_list = [genre.strip() for genre in genres.split(',')]
    for genre in genre_list:
        if genre in genre_count:
            genre_count[genre] += 1
        else:
            genre_count[genre] = 1

# Print the number of unique genres and their occurrences
print(f"Number of Genres: {len(genre_count)}")
print(genre_count)

Number of Genres: 23
{'Action': 2946, 'Adventure': 2113, 'Drama': 6875, 'Comedy': 4367, 'Thriller': 2232, 'Mystery': 1308, 'Family': 766, 'Fantasy': 962, 'Romance': 2140, 'Biography': 807, 'History': 441, 'Sport': 252, 'Crime': 2222, 'Musical': 200, 'Sci-fi': 1816, 'Horror': 1816, 'Animation': 565, 'Music': 352, 'War': 259, 'Documentary': 192, 'Western': 176, 'Film': 52, 'Noir': 52}


In [14]:
# Set a threshold for identifying rare genres based on a percentage of the dataset size
threshold = int(len(df) * 0.01)

# Identify genres with occurrences below the threshold
rare_genres = [key for key, value in genre_count.items() if value < threshold]

# Print the number of rare genres and display the first 5
print(f"Number of Rare Genres: {len(rare_genres)}")
print(rare_genres[:5])

Number of Rare Genres: 2
['Film', 'Noir']


In [15]:
# Extract 'genres' column values as a list
genres_list = df.genres.to_list()

# Initialize lists to store revised genres and indices to drop
revised_genre_list = []
indices_to_drop = []

# Iterate through each row in the 'genres' column
for idx, genres in enumerate(genres_list):
    # Split genres and remove extra spaces
    genre_list = [genre.strip() for genre in genres.split(',')]

    # Filter out rare genres
    revised_genres = [genre for genre in genre_list if genre not in rare_genres]

    # Check if no valid genres remain after filtering
    if len(revised_genres) == 0:
        indices_to_drop.append(idx)
    else:
        revised_genre_list.append(revised_genres)

# Drop rows with no valid genres and reset the index
df = df.drop(indices_to_drop).reset_index(drop=True)

# Display the shape of the DataFrame after filtering
df.shape

(12874, 3)

In [16]:
# Add a new column 'revised_genres' to the DataFrame with the filtered genre lists
df['revised_genres'] = revised_genre_list

In [17]:
# Extract 'revised_genres' column values as a list
revised_genres_list = df.revised_genres.to_list()

# Count the occurrences of each genre in the revised genre list
revised_genre_count = {}
for genres in revised_genres_list:
    for genre in genres:
        if genre in revised_genre_count:
            revised_genre_count[genre] += 1
        else:
            revised_genre_count[genre] = 1

# Print the number of unique genres and their occurrences after filtering
print(f"Number of Genres: {len(revised_genre_count)}")
print(revised_genre_count)

Number of Genres: 21
{'Action': 2946, 'Adventure': 2113, 'Drama': 6875, 'Comedy': 4367, 'Thriller': 2232, 'Mystery': 1308, 'Family': 766, 'Fantasy': 962, 'Romance': 2140, 'Biography': 807, 'History': 441, 'Sport': 252, 'Crime': 2222, 'Musical': 200, 'Sci-fi': 1816, 'Horror': 1816, 'Animation': 565, 'Music': 352, 'War': 259, 'Documentary': 192, 'Western': 176}


In [18]:
# Create a dictionary to encode genre types as numerical indices
encode_genre_types = {key: idx for idx, (key, value) in enumerate(revised_genre_count.items())}

# Save the encoded genre types dictionary to a JSON file
with open("genre_types_encoded.json", "w") as fp:
    json.dump(encode_genre_types, fp)

In [19]:
# Extract labels (genre types) from the encoded genre types dictionary
labels = list(encode_genre_types.keys())

# Print the number of labels and display the first 5
print(f"Number of Labels: {len(labels)}")
print(labels[:5])

Number of Labels: 21
['Action', 'Adventure', 'Drama', 'Comedy', 'Thriller']


In [20]:
# Initialize an empty list to store categorical genre representations
categorical_genre_list = []

# Extract 'revised_genres' column values as a list
revised_genres_list = df.revised_genres.to_list()

# Iterate through each row in the 'revised_genres' column
for revised_genres in revised_genres_list:
    # Initialize a list of zeros for each possible genre type
    categorical_list = [0] * len(encode_genre_types)

    # Update the list with 1 for each genre present in the current row
    for genre in revised_genres:
        genre_type_index = encode_genre_types[genre]
        categorical_list[genre_type_index] = 1

    # Append the categorical list to the main list
    categorical_genre_list.append(categorical_list)

# Add a new column 'genre_cat_list' to the DataFrame with categorical genre representations
df['genre_cat_list'] = categorical_genre_list

# Display the first 23 elements of the categorical representation for the 4th row
categorical_genre_list[3][:23]

[0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

##**Data Split**

In [21]:
# Initialize a RandomSplitter with a validation percentage of 10% and seed 42
splitter = RandomSplitter(valid_pct=0.1, seed=42)

# Split the DataFrame indices into training and validation sets
train_ids, valid_ids = splitter(df)

# Print the number of samples in the training and validation sets
print(f"Number of samples in training set: {len(train_ids)}")
print(f"Number of samples in validation set: {len(valid_ids)}")

Number of samples in training set: 11587
Number of samples in validation set: 1287


In [22]:
# Create a DataFrame containing samples from the validation set
valid_df = df.loc[valid_ids]

# Display the first 2 rows of the validation DataFrame
valid_df.head(2)

,Title,Description,genres,revised_genres,genre_cat_list
5494,Dead Man's Shoes,A disaffected soldier returns to his hometown to get even with the thugs who brutalized his mentally-challenged brother years ago.,"Crime, Drama, Thriller","[Crime, Drama, Thriller]","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
10461,Nothing Underneath,"A park ranger from Wyoming, whose twin sister is a fashion model in Milan, Italy, strongly senses that his sister is in mortal danger. Once he arrives in Milan, he discovers that his sister has disappeared.","Mystery, Thriller","[Mystery, Thriller]","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


##**Fastai and Blurr Inference**

In [23]:
# Specify the path to the trained model file
model_path = "models/multi-movie_genre-classifier-stage-2.pkl"

# Load the model and associated information using fastai's load_learner function
learner_inf = load_learner(model_path)

In [24]:
# Use the loaded learner to make predictions on a text input
prediction = learner_inf.blurr_predict("I have promises to keep")[0]['labels']

# Display the predicted labels for the input text
print(prediction)

['Drama', 'Comedy']


##**Evaluation**

In [25]:
from sklearn import metrics

def metric_measures(test_df, preds):
    """
    Calculate and print evaluation metrics for the model predictions.

    Args:
    - test_df (DataFrame): DataFrame containing the ground truth labels.
    - preds (List): List of model predictions.

    Returns:
    - None
    """

    # Convert 'genre_cat_list' column in the DataFrame to numpy arrays
    targets = [np.asarray(target) for target in test_df['genre_cat_list'].to_list()]
    outputs = [np.asarray(pred) for pred in preds]

    # Calculate accuracy, F1 score (micro), and F1 score (macro)
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

    # Print the calculated metrics
    print(f"Accuracy = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

    return

In [26]:
# Initialize an empty list to store model predictions
preds = []

# Iterate through each row in the validation DataFrame
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
    # Get the 'Description' from the current row
    desc = row['Description']

    # Make predictions using the loaded learner for the current 'Description'
    labels = learner_inf.blurr_predict(desc)[0]['labels']

    # Initialize a list of zeros for each possible genre type
    pred_genres = [0] * len(encode_genre_types)

    # Update the list with 1 for each predicted genre
    for label in labels:
        pred_genres[encode_genre_types[label]] = 1

    # Append the list to the main list of predictions
    preds.append(pred_genres)

# Display the first 20 elements of the predicted genre representation for the first row
preds[0][:20]

  0%|          | 0/1287 [00:00<?, ?it/s]

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [27]:
# Evaluate and print metric measures using the defined function
metric_measures(valid_df, preds)

Accuracy = 0.12665112665112666
F1 Score (Micro) = 0.5879571248423708
F1 Score (Macro) = 0.4969601500695055


##**Convert to ONNX**

In [29]:
# Specify the path to the trained model file (stage 3)
model_path = "models/multi-movie_genre-classifier-stage-2.pkl"

# Load the model and associated information using fastai's load_learner function
learner_inf = load_learner(model_path)

In [30]:
# Set the classifier to evaluation mode
classifier = learner_inf.model.hf_model.eval()

# Export the model to ONNX format
torch.onnx.export(
    classifier,  # PyTorch model
    torch.LongTensor([[0] * 512]),  # Sample input (batch_size=1, sequence_len=512)
    'models/movie_genre-classifier.onnx',  # Output ONNX file path
    input_names=['input_ids'],  # Input names
    output_names=['output'],  # Output names
    opset_version=13,  # ONNX opset version
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},  # Dynamic axes for input
        'output': {0: 'batch_size'}  # Dynamic axes for output
    }
)

In [31]:
from onnxruntime.quantization import quantize_dynamic, QuantType

# Specify the paths for the original and quantized ONNX models
onnx_model_path = 'models/movie_genre-classifier.onnx'
quantized_onnx_model_path = 'models/movie_genre-classifier-quantized.onnx'

# Quantize the original ONNX model and save the quantized model
quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

## **ONNX Inference**


### **Normal ONNX**

In [32]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

# Get the class labels from the encoded genre types dictionary
class_labels = list(encode_genre_types.keys())

# Load the ONNX inference session
inf_session = rt.InferenceSession('models/movie_genre-classifier.onnx')

# Get the input and output names from the ONNX model
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [34]:
# Initialize an empty list to store model predictions
preds = []

# Iterate through each row in the validation DataFrame
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
    # Get the 'Description' from the current row
    desc = row['Description']

    # Tokenize the description and get input IDs (truncate to max length of 512)
    input_ids = tokenizer(desc)['input_ids'][:512]

    # Run the ONNX model to obtain probabilities
    probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
    probs = torch.FloatTensor(probs)

    # Apply sigmoid threshold and get labels based on masks
    masks = torch.sigmoid(probs) >= 0.5
    labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

    # Initialize a list of zeros for each possible genre type
    pred_genres = [0] * len(encode_genre_types)

    # Update the list with 1 for each predicted genre
    for label in labels:
        pred_genres[encode_genre_types[label]] = 1

    # Append the list to the main list of predictions
    preds.append(pred_genres)

  0%|          | 0/1287 [00:00<?, ?it/s]

In [35]:
# Evaluate and print metric measures using the defined function
metric_measures(valid_df, preds)

Accuracy = 0.128982128982129
F1 Score (Micro) = 0.5890713834491471
F1 Score (Macro) = 0.49850730577978936


###**Quantized ONNX**

In [36]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

# Get the class labels from the encoded genre types dictionary
class_labels = list(encode_genre_types.keys())

# Load the quantized ONNX inference session
inf_session = rt.InferenceSession('models/movie_genre-classifier-quantized.onnx')

# Get the input and output names from the quantized ONNX model
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [37]:
# Initialize an empty list to store model predictions
preds = []

# Iterate through each row in the validation DataFrame
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
    # Get the 'Description' from the current row
    desc = row['Description']

    # Tokenize the description and get input IDs (truncate to max length of 512)
    input_ids = tokenizer(desc)['input_ids'][:512]

    # Run the quantized ONNX model to obtain probabilities
    probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
    probs = torch.FloatTensor(probs)

    # Apply sigmoid threshold and get labels based on masks
    masks = torch.sigmoid(probs) >= 0.5
    labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

    # Initialize a list of zeros for each possible genre type
    pred_genres = [0] * len(encode_genre_types)

    # Update the list with 1 for each predicted genre
    for label in labels:
        pred_genres[encode_genre_types[label]] = 1

    # Append the list to the main list of predictions
    preds.append(pred_genres)

  0%|          | 0/1287 [00:00<?, ?it/s]

In [38]:
# Evaluate and print metric measures using the defined function
metric_measures(valid_df, preds)

Accuracy = 0.13364413364413363
F1 Score (Micro) = 0.589666613999052
F1 Score (Macro) = 0.49817876635685


In [39]:
# Save the DataFrame to a CSV file named "movie_data_cleaned.csv"
df.to_csv('movie_data_cleaned.csv', index=False)